In [19]:
#annoy 패키지를 설치합니다.
!pip install annoy

In [20]:
#사전 훈련된 단어 임베딩 사용하기
import torch
import torch.nn as nn
from tqdm import tqdm
from annoy import AnnoyIndex
import numpy as np

In [21]:
#단어 임베딩을 사용한 유추 작업
class PreTrainedEmbeddings(object):
    """ 사전 훈련된 단어 벡터 사용을 위한 래퍼 클래스 """
    def __init__(self, word_to_index, word_vectors):
        """
        매개변수:
            word_to_index (dict): 단어에서 정수로 매핑
            word_vectors (numpy 배열의 리스트)
        """
        self.word_to_index = word_to_index
        self.word_vectors = word_vectors
        self.index_to_word = {v: k for k, v in self.word_to_index.items()}

        self.index = AnnoyIndex(len(word_vectors[0]), metric='euclidean')
        print("인덱스 만드는 중!")
        for _, i in self.word_to_index.items():
            self.index.add_item(i, self.word_vectors[i])
        self.index.build(50)
        print("완료!")
        
    @classmethod
    def from_embeddings_file(cls, embedding_file):
        """사전 훈련된 벡터 파일에서 객체를 만듭니다.
        
        벡터 파일은 다음과 같은 포맷입니다:
            word0 x0_0 x0_1 x0_2 x0_3 ... x0_N
            word1 x1_0 x1_1 x1_2 x1_3 ... x1_N
        
        매개변수:
            embedding_file (str): 파일 위치
        반환값:
            PretrainedEmbeddings의 인스턴스
        """
        word_to_index = {}
        word_vectors = []

        with open(embedding_file) as fp:
            for line in fp.readlines():
                line = line.split(" ")
                word = line[0]
                vec = np.array([float(x) for x in line[1:]])
                
                word_to_index[word] = len(word_to_index)
                word_vectors.append(vec)
                
        return cls(word_to_index, word_vectors)
    
    def get_embedding(self, word):
        """
        매개변수:
            word (str)
        반환값
            임베딩 (numpy.ndarray)
        """
        return self.word_vectors[self.word_to_index[word]]

    def get_closest_to_vector(self, vector, n=1):
        """벡터가 주어지면 n 개의 최근접 이웃을 반환합니다
        매개변수:
            vector (np.ndarray): Annoy 인덱스에 있는 벡터의 크기와 같아야 합니다
            n (int): 반환될 이웃의 개수
        반환값:
            [str, str, ...]: 주어진 벡터와 가장 가까운 단어
                단어는 거리순으로 정렬되어 있지 않습니다.
        """
        nn_indices = self.index.get_nns_by_vector(vector, n)
        return [self.index_to_word[neighbor] for neighbor in nn_indices]
    
    def compute_and_print_analogy(self, word1, word2, word3):
        """단어 임베딩을 사용한 유추 결과를 출력합니다

        word1이 word2일 때 word3은 __입니다.
        이 메서드는 word1 : word2 :: word3 : word4를 출력합니다
        
        매개변수:
            word1 (str)
            word2 (str)
            word3 (str)
        """
        vec1 = self.get_embedding(word1)
        vec2 = self.get_embedding(word2)
        vec3 = self.get_embedding(word3)

        # 네 번째 단어 임베딩을 계산합니다
        spatial_relationship = vec2 - vec1
        vec4 = vec3 + spatial_relationship

        closest_words = self.get_closest_to_vector(vec4, n=4)
        existing_words = set([word1, word2, word3])
        closest_words = [word for word in closest_words 
                             if word not in existing_words] 

        if len(closest_words) == 0:
            print("계산된 벡터와 가장 가까운 이웃을 찾을 수 없습니다!")
            return
        
        for word4 in closest_words:
            print("{} : {} :: {} : {}".format(word1, word2, word3, word4))

In [22]:
# GloVe 데이터를 다운로드합니다.
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip
!mkdir -p data/glove
!mv glove.6B.100d.txt data/glove

--2022-02-03 14:03:07--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-02-03 14:03:07--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-02-03 14:03:07--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove

In [23]:
embeddings = PreTrainedEmbeddings.from_embeddings_file('data/glove/glove.6B.100d.txt')

인덱스 만드는 중!
완료!


In [24]:
# SAT 유추 작업에서 봤듯이 단어 임베딩은 많은 언어 관계를 인코딩합니다.
# 관계 1: 성별 명사와 대명사의 관계
embeddings.compute_and_print_analogy('man', 'he', 'woman')

man : he :: woman : she
man : he :: woman : her


In [25]:
# 관계 2: 동사-명사 관계
embeddings.compute_and_print_analogy('fly', 'plane', 'sail')

fly : plane :: sail : ship
fly : plane :: sail : vessel
fly : plane :: sail : boat


In [26]:
# 관계 3: 명사-명사 관계
embeddings.compute_and_print_analogy('cat', 'kitten', 'dog')

cat : kitten :: dog : puppy
cat : kitten :: dog : puppies
cat : kitten :: dog : junkyard


In [27]:
# 관계 4: 상위어(Hypernymy) (더 넓은 범주)
embeddings.compute_and_print_analogy('blue', 'color', 'dog')

blue : color :: dog : animal
blue : color :: dog : pet
blue : color :: dog : taste
blue : color :: dog : touch


In [28]:
# 관계 5: 부분에서 전체(Meronymy)
embeddings.compute_and_print_analogy('toe', 'foot', 'finger')

toe : foot :: finger : hand
toe : foot :: finger : kept
toe : foot :: finger : ground


In [29]:
# 관계 6: 방식 차이(Troponymy) 
embeddings.compute_and_print_analogy('talk', 'communicate', 'read')

talk : communicate :: read : interpret
talk : communicate :: read : communicated
talk : communicate :: read : transmit


In [30]:
# 관계 7: 전체 의미 표현(Metonymy) (관습 / 인물)
embeddings.compute_and_print_analogy('blue', 'democrat', 'red')

blue : democrat :: red : republican
blue : democrat :: red : congressman
blue : democrat :: red : senator


In [31]:
# 관계 8: 비교급
embeddings.compute_and_print_analogy('fast', 'fastest', 'young')

fast : fastest :: young : younger
fast : fastest :: young : sixth
fast : fastest :: young : fifth
fast : fastest :: young : seventh


이런 관계는 언어의 작동 방식에 대해 체계적인 것처럼 보이지만 문제가 발생할 수 있습니다. 


밑에 등장하는 코드에서 볼 수 있듯이 단어 벡터는 동시에 등장하는 정보를 기반으로 하므로 잘못된 관계가 만들어지기도 합니다.

In [32]:
#동시에 등장하는 정보로 의미를 인코딩하는 위험을 보여주는 예시입니다.. 항상 이렇지는 않다고 합니다!
embeddings.compute_and_print_analogy('fast', 'fastest', 'small')

fast : fastest :: small : smallest
fast : fastest :: small : largest
fast : fastest :: small : among
fast : fastest :: small : quarters


밑에 등장할 코드는 널리 알려진 단어 임베딩의 쌍인 성별 인코딩을 보여 줍니다.

In [33]:
#단어 임베딩에 인코딩된 성별과 같은 보호 속성에 주의하세요. 이로 인해 하위 모델에서 원치 않는 편향이 발생할 수 있습니다.
embeddings.compute_and_print_analogy('man', 'king', 'woman')

man : king :: woman : queen
man : king :: woman : monarch
man : king :: woman : throne


언어 규칙과 성문화된 문화 편견은 구별하기 어렵습니다.

예를 들어 의사는 사실상 남성이 아니고 간호사는 여성이 아니지만 이런 오랜 문화 편견은 언어의 규칙성으로 관찰되고 밑에 나올 코드처럼 단어 벡터로 나타납니다.

In [34]:
#벡터에 인코딩된 문화적 성별 편견
embeddings.compute_and_print_analogy('man', 'doctor', 'woman')

man : doctor :: woman : nurse
man : doctor :: woman : physician
man : doctor :: woman : doctors
